<a href="https://colab.research.google.com/github/vincentungu00/Restaurant-Orders/blob/main/Restaurant_Orders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [153]:
import pandas as pd
import numpy as np
import datetime

pd.options.mode.copy_on_write = True

In [145]:
# Load the data sets

menu_items = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/main/Sources/menu_items.csv')
order_details = pd.read_csv('https://raw.githubusercontent.com/vincentungu00/Restaurant-Orders/main/Sources/order_details.csv')

In [146]:
menu_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   menu_item_id  32 non-null     int64  
 1   item_name     32 non-null     object 
 2   category      32 non-null     object 
 3   price         32 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ KB


In [147]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12234 entries, 0 to 12233
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_details_id  12234 non-null  int64  
 1   order_id          12234 non-null  int64  
 2   order_date        12234 non-null  object 
 3   order_time        12234 non-null  object 
 4   item_id           12097 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 478.0+ KB


In [148]:
# Process menu_items
menu_items_processed = menu_items.copy().drop_duplicates()

In [160]:
# Process order_details
def get_day(date_str):
  day_of_week = date_str.strftime('%A')
  weekflag = 'Weekend' if day_of_week in ['Saturday', 'Sunday'] else 'Weekday'
  return day_of_week, weekflag

def get_eat_time(hour_str):
  if 6 <= hour_str < 9:
    return 'Breakfast'
  elif 9 <= hour_str < 12:
    return 'Brunch'
  elif 12 <= hour_str < 14:
    return 'Lunch'
  elif 14 <= hour_str < 18:
    return 'Afternoon Snack'
  elif 18 <= hour_str < 20:
    return 'Dinner'
  elif 20 <= hour_str < 23:
    return 'Evening Snack'
  else:
    return 'Late Night'

order_details_deduplicate = order_details.copy().drop_duplicates()
order_details_processed = order_details_deduplicate.dropna()

order_details_processed['order_details_id'] = order_details_processed.groupby(by=['order_id', 'order_date', 'order_time'])['order_details_id'].transform(lambda x: x.rank()).astype('int64')
order_details_processed['order_date'] = pd.to_datetime(order_details_processed['order_date'], format='%m/%d/%y')
order_details_processed['order_date_day'] = order_details_processed['order_date'].dt.day
order_details_processed['order_date_month'] = order_details_processed['order_date'].dt.month
order_details_processed['order_date_year'] = order_details_processed['order_date'].dt.year
order_details_processed['order_time'] = pd.to_datetime(order_details_processed['order_time'], format='%I:%M:%S %p').dt.time

order_details_processed['order_time_hour'] = order_details_processed['order_time'].apply(lambda x: x.hour)
order_details_processed['order_time_minute'] = order_details_processed['order_time'].apply(lambda x: x.minute)
order_details_processed['order_time_second'] = order_details_processed['order_time'].apply(lambda x: x.second)

order_details_processed[['order_date_day_name', 'order_date_weekflag']] = order_details_processed['order_date'].apply(lambda x: get_day(x)).tolist()
order_details_processed['order_eat_time'] = order_details_processed['order_time_hour'].apply(lambda x: get_eat_time(x))

order_details_processed['item_id'] = order_details_processed['item_id'].astype('int64')

del order_details_deduplicate

In [161]:
order_details_processed

,order_details_id,order_id,order_date,order_time,item_id,order_date_day,order_date_month,order_date_year,order_time_hour,order_time_minute,order_time_second,order_date_day_name,order_date_weekflag,order_eat_time
0,1,1,2023-01-01,11:38:36,109,1,1,2023,11,38,36,Sunday,Weekend,Brunch
1,1,2,2023-01-01,11:57:40,108,1,1,2023,11,57,40,Sunday,Weekend,Brunch
2,2,2,2023-01-01,11:57:40,124,1,1,2023,11,57,40,Sunday,Weekend,Brunch
3,3,2,2023-01-01,11:57:40,117,1,1,2023,11,57,40,Sunday,Weekend,Brunch
4,4,2,2023-01-01,11:57:40,129,1,1,2023,11,57,40,Sunday,Weekend,Brunch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12229,1,5369,2023-03-31,22:05:04,109,31,3,2023,22,5,4,Friday,Weekday,Evening Snack
12230,2,5369,2023-03-31,22:05:04,129,31,3,2023,22,5,4,Friday,Weekday,Evening Snack
12231,3,5369,2023-03-31,22:05:04,120,31,3,2023,22,5,4,Friday,Weekday,Evening Snack
12232,4,5369,2023-03-31,22:05:04,122,31,3,2023,22,5,4,Friday,Weekday,Evening Snack


In [163]:
orders = order_details_processed.merge(menu_items_processed, left_on='item_id', right_on='menu_item_id', how='left')
orders['revenue'] = orders['price'].apply(lambda x: x * 0.5)
orders.drop(columns=['item_id'], inplace=True)
orders

,order_details_id,order_id,order_date,order_time,order_date_day,order_date_month,order_date_year,order_time_hour,order_time_minute,order_time_second,order_date_day_name,order_date_weekflag,order_eat_time,menu_item_id,item_name,category,price,revenue
0,1,1,2023-01-01,11:38:36,1,1,2023,11,38,36,Sunday,Weekend,Brunch,109,Korean Beef Bowl,Asian,17.95,8.975
1,1,2,2023-01-01,11:57:40,1,1,2023,11,57,40,Sunday,Weekend,Brunch,108,Tofu Pad Thai,Asian,14.50,7.250
2,2,2,2023-01-01,11:57:40,1,1,2023,11,57,40,Sunday,Weekend,Brunch,124,Spaghetti,Italian,14.50,7.250
3,3,2,2023-01-01,11:57:40,1,1,2023,11,57,40,Sunday,Weekend,Brunch,117,Chicken Burrito,Mexican,12.95,6.475
4,4,2,2023-01-01,11:57:40,1,1,2023,11,57,40,Sunday,Weekend,Brunch,129,Mushroom Ravioli,Italian,15.50,7.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12092,1,5369,2023-03-31,22:05:04,31,3,2023,22,5,4,Friday,Weekday,Evening Snack,109,Korean Beef Bowl,Asian,17.95,8.975
12093,2,5369,2023-03-31,22:05:04,31,3,2023,22,5,4,Friday,Weekday,Evening Snack,129,Mushroom Ravioli,Italian,15.50,7.750
12094,3,5369,2023-03-31,22:05:04,31,3,2023,22,5,4,Friday,Weekday,Evening Snack,120,Steak Torta,Mexican,13.95,6.975
12095,4,5369,2023-03-31,22:05:04,31,3,2023,22,5,4,Friday,Weekday,Evening Snack,122,Chips & Salsa,Mexican,7.00,3.500


In [164]:
orders.to_csv('orders.csv', index=False)
menu_items_processed.to_csv('menu_items_processed.csv', index=False)
order_details_processed.to_csv('order_details_processed.csv', index=False)